In [ ]:
import math
import numpy as np

In [48]:
dataset = [
	[2.771244718, 1.784783929, 0],
	[1.728571309, 1.169761413, 0],
	[3.678319846, 2.81281357, 0],
	[3.961043357, 2.61995032, 0],
	[2.999208922, 2.209014212, 0],
	[7.497545867, 3.162953546, 1],
	[9.00220326, 3.339047188, 1],
	[7.444542326, 0.476683375, 1],
	[10.12493903, 3.234550982, 1],
	[6.642287351, 3.319983761, 1]
]
# inspired by https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/

In [49]:
# Calculate the Gini index for a split dataset --> The lower it is, the better the data has been split.
def gini_index(groups, classes):
	# count all samples at split point
	n_instances = float(sum([len(group) for group in groups]))
	# sum weighted Gini index for each group
	gini = 0.0
	for group in groups:
		size = float(len(group)) # You could divide it into more than 2 groups, a non-binary tree (less common).
		# avoid divide by zero
		if size == 0:
			continue
		score = 0.0
		# score the group based on the score for each class
		for class_val in classes:
			p = [row[-1] for row in group].count(class_val) / size
			score += p * p
		# weighting the group score by its relative size
		gini += (1.0 - score) * (size / n_instances)
	return gini

In [50]:
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
	left, right = list(), list()
	for row in dataset:
		if row[index] < value: # Convention: left is smaller and right is larger.
			left.append(row)
		else:
			right.append(row)
	return left, right

# Select the best split point for a dataset
def get_split(dataset):
	class_values = list(set(row[-1] for row in dataset))
	b_index, b_value, b_groups, b_score = None, None, None, math.inf
	for index in range(len(dataset[0]) - 1): # The last feature is the class, in these datasets - do not consider it.
		for row in dataset:
			groups = test_split(index, row[index], dataset) # Splitting on the data already present in the dataset, efficient.
			gini = gini_index(groups, class_values)
			if gini < b_score:
				b_index, b_value, b_score, b_groups = index, row[index], gini, groups
	return {'index': b_index, 'value': b_value, 'groups': b_groups} # This is the best split, you apply it recursively.

# Create a terminal node value
def to_terminal(group): # I set the most frequently occurring class in the last split as the final class.
	outcomes = [row[-1] for row in group] # The class is in the last feature of the row, denoted by row[-1].

	n_class = 2
	counter = [0 for _ in range(n_class)]
	for out in outcomes:
		counter[out] += 1
	return np.argmax(counter)

# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth): # You don't return anything, but recursively modify the root (parameter node).
	left, right = node['groups']
	del(node['groups']) # They won't be needed anymore, you'll only use them to split during training.
	# check for a no split
	if not left or not right:
		node['left'] = node['right'] = to_terminal(left + right)
		return
	# check for max depth
	if depth >= max_depth:
		node['left'], node['right'] = to_terminal(left), to_terminal(right)
		return
	# process left child
	if len(left) <= min_size:
		node['left'] = to_terminal(left)
	else:
		node['left'] = get_split(left)
		split(node['left'], max_depth, min_size, depth + 1)
	# process right child
	if len(right) <= min_size:
		node['right'] = to_terminal(right)
	else:
		node['right'] = get_split(right)
		split(node['right'], max_depth, min_size, depth + 1)

# Build a decision tree
def build_tree(train, max_depth, min_size):
	root = get_split(train)
	split(root, max_depth, min_size, 1)
	return root

# Print a decision tree
def print_tree(node, depth=0):
	if isinstance(node, dict):
		print('%s[X%d < %.3f]' % (depth * '  ', (node['index'] + 1), node['value']))
		print_tree(node['left'], depth + 1)
		print_tree(node['right'], depth + 1)
	else:
		print('%s[%s]' % (depth * '  ', node))

tree = build_tree(dataset, 3, 1)

In [51]:
# Make a prediction with a decision tree --> You could easily perform multi-class classification as well.
def predict(node, row):
	if row[node['index']] < node['value']:
		if isinstance(node['left'], dict):
			return predict(node['left'], row)
		else:
			return node['left']
	else:
		if isinstance(node['right'], dict):
			return predict(node['right'], row)
		else:
			return node['right']

In [52]:
stump = {'index': 0, 'right': 1, 'value': 6.642287351, 'left': 0} # feature, pred_greater, split_val, pred_smaller
# This is the structure of a terminal node of the tree - if 'right' and 'left' were nodes themselves, they would be intermediate nodes.
prediction = predict(tree, dataset[0][0:2])
print('Expected = %d, Got = %d' % (dataset[0][-1], prediction))
# An example of prediction on the first sample.

Expected = 0, Got = 0


In [53]:
'''Here is an example of a decision tree classifier - it works well for these simple datasets (both numeric and categorical). The main advantages are explainability and scalability. It can be enhanced with Bagging (to reduce Overfitting) and Boosting (to reduce Underfitting). The most famous example of Bagging is the Random Forest algorithm - it combines (through averaging) the decisions of N decision trees.'''

In [54]:
print_tree(tree) # with this data you can draw the decision tree and understand the subdivisions
# In this case, it is evident that the variable X2 is not useful - you consistently chose to split on X1.

[X1 < 6.642]
  [X1 < 2.771]
    [0]
    [X1 < 2.771]
      [0]
      [0]
  [X1 < 7.498]
    [X1 < 7.445]
      [1]
      [1]
    [X1 < 7.498]
      [1]
      [1]
